In [5]:
# configurations start
MODEL_FILE = 'deploy.prototxt'
PRETRAINED = 'minc.caffemodel'
imagePrefix="../../0_get_data/data/images_"
imageDirs=[imagePrefix+'texture/', imagePrefix+'bak/', imagePrefix+'whole/'];
imageFeaturePath='minc_features.pkl'
# configurations end

import caffe
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

!pwd

caffe.set_mode_gpu()
net = caffe.Net(MODEL_FILE, PRETRAINED, caffe.TEST)

in_=net.inputs[0]
transformer = caffe.io.Transformer({in_ : net.blobs[in_].data.shape})
transformer.set_mean(in_, mean=np.array([104,117,124]))
transformer.set_raw_scale(in_, 255)
transformer.set_channel_swap(in_, (2,1,0))
transformer.set_transpose(in_, (2, 0, 1))

crop_dims=(224, 224);

output_blobs=['prob', 'pool5/7x7_s1', 'inception_5a/output', \
              'pool4/3x3_s2', 'inception_4c/output', 'inception_4a/output'];

def getActivations(inputs):
    # inputs is the concatenate of images
    input_ = np.zeros((len(inputs), 3, crop_dims[0], crop_dims[1]), dtype=np.float32)
    for ix, in_ in enumerate(inputs):
        input_[ix]=transformer.preprocess(net.inputs[0], caffe.io.resize_image(in_, crop_dims) )
        
    out = net.forward_all(output_blobs, **{net.inputs[0]: input_})
    
    return out

/home/yang/publish_release_test/multimodal/2_image/2_getImageFeatures


In [6]:
def processDir(dirName):
    files=!ls $dirName
    inputs=[]
    
    print 'start'+dirName
    for f in files:
        if len(f)>4 and (not f.startswith('.')) and f.endswith('.jpg'):
            #print f
            input_image = caffe.io.load_image(dirName+f)
            inputs.append(input_image);
    
    print 'read images done!'
    out=getActivations(inputs)
    
    print 'GPU computing done!'
    return out

In [7]:
import pickle
def extractAllFeatures():
    folders=imageDirs;

    # output format: 
    # first list level: diff scopes; second list level: diff layers
    # each element is a np array
    outs=[];
    for folder in folders:
        outs.append(processDir(folder))
    return outs

In [8]:
outs=extractAllFeatures()

start../../0_get_data/data/images_texture/
read images done!
GPU computing done!
start../../0_get_data/data/images_bak/
read images done!
GPU computing done!
start../../0_get_data/data/images_whole/
read images done!
GPU computing done!


In [9]:
pickle.dump(outs, open(imageFeaturePath,'w'), pickle.HIGHEST_PROTOCOL)